In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
# Spark session & context
#spark = (SparkSession
#         .builder
#         .master('local[*]')
#         .appName('kafka')
         # Add kafka package
         #.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
#         .getOrCreate())
#sc = spark.sparkContext

In [2]:
spark

In [3]:
# Create stream dataframe setting kafka server, topic and offset option
df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka-broker:9092") # kafka server
  .option("subscribe", "teste1") # topic
  .option("startingOffsets", "earliest") # start from beginning 
  #.option("kafka.group.id", "spark2")
  .load())

In [4]:
df

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [7]:
ds = df.selectExpr("CAST(value AS STRING)")

In [8]:
print(type(df))
print(type(ds))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
rawQuery = df \
        .writeStream \
        .queryName("qraw")\
        .format("memory")\
        .start()

In [10]:
alertQuery = ds \
        .writeStream \
        .queryName("qalerts")\
        .format("memory")\
        .start()

In [11]:
raw = spark.sql("select * from qraw")
raw.show()

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



In [12]:
raw.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [19]:
alerts = spark.sql("select * from qalerts")
alerts.show()

+--------------------+
|               value|
+--------------------+
| "teste teste teste"|
| "teste teste teste"|
| "teste teste teste"|
| "teste teste teste"|
| "teste teste teste"|
| "teste teste teste"|
| "teste teste teste"|
| "teste teste teste"|
|"teste teste test...|
|"teste teste test...|
|"teste teste test...|
|              "aaaa"|
|               "bbb"|
|               "ccc"|
|               "ddd"|
|               "eee"|
+--------------------+



In [18]:
ds \
    .writeStream \
    .trigger(once=True) \
    .format("console") \
    .outputMode("append") \
    .start() \
    .awaitTermination()